In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from tqdm import tqdm
tqdm.pandas() # для работы progress_apply в пандас

from sentence_transformers import SentenceTransformer, util
from concurrent import futures
from tqdm import tqdm
import pymorphy2

import warnings
warnings.filterwarnings('ignore')

# pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_colwidth', None)
sns.set(rc={'figure.figsize':(18,8)}, style="darkgrid")

## Загрузка и знакомство с данными

from sklearn.neighbors import NearestNeighbors
import random
import math
import pylab as pl
import numpy as np
from matplotlib.colors import ListedColormap

In [2]:
# загрузка датасетов
DATA_DIR = 'data/'
try:
    dealers_id = pd.read_csv(DATA_DIR + 'marketing_dealer.csv', sep=";")
    data_dealers = pd.read_csv(DATA_DIR +'marketing_dealerprice.csv', sep=";", parse_dates=['date'])
    data_products = pd.read_csv(DATA_DIR + 'marketing_product.csv', sep=";", index_col='Unnamed: 0',)
    data_key = pd.read_csv(DATA_DIR + 'marketing_productdealerkey.csv',  sep=";")
except:
    delers_id = pd.read_csv('marketing_dealer.csv', sep=";")
    data_dealers = pd.read_csv('marketing_dealerprice.csv', sep=";", parse_dates=['date'])
    data_products = pd.read_csv('marketing_product.csv', sep=";", index_col='Unnamed: 0',)
    data_key  = pd.read_csv('marketing_productdealerkey.csv', sep=';')

In [3]:
data_key = data_key[data_key['key'].isin(data_dealers.product_key)]

In [4]:
data_key.head()

,id,key,dealer_id,product_id
0,1,546227,2,12
1,2,651265,2,106
2,3,546257,2,200
3,4,546408,2,38
4,5,651258,2,403


In [5]:
data_dealers.head()

,id,product_key,price,product_url,product_name,date,dealer_id
0,2,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_prosept_universal_spray_500ml/,"Средство универсальное Prosept Universal Spray, 500мл",2023-07-11,2
1,3,546408,175.0,https://akson.ru//p/kontsentrat_prosept_multipower_dlya_mytya_polov_tsitrus_1l/,"Концентрат Prosept Multipower для мытья полов, цитрус 1л",2023-07-11,2
2,4,546234,285.0,https://akson.ru//p/sredstvo_dlya_chistki_lyustr_prosept_universal_anti_dust_500ml/,"Средство для чистки люстр Prosept Universal Anti-dust, 500мл",2023-07-11,2
3,5,651258,362.0,https://akson.ru//p/udalitel_rzhavchiny_prosept_rust_remover_0_5l_023_05/,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 023-05",2023-07-11,2
4,6,546355,205.0,https://akson.ru//p/sredstvo_moyushchee_dlya_bani_i_sauny_prosept_multipower_wood_1l/,Средство моющее для бани и сауны Prosept Multipower Wood 1л,2023-07-11,2


In [6]:
data_dealers = data_dealers[['product_key','product_name', 'dealer_id']]
data_dealers.head()

,product_key,product_name,dealer_id
0,546227,"Средство универсальное Prosept Universal Spray, 500мл",2
1,546408,"Концентрат Prosept Multipower для мытья полов, цитрус 1л",2
2,546234,"Средство для чистки люстр Prosept Universal Anti-dust, 500мл",2
3,651258,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 023-05",2
4,546355,Средство моющее для бани и сауны Prosept Multipower Wood 1л,2


In [7]:
data_products = data_products[['article','name_1c']]
data_products.head()

,article,name_1c
0,008-1,"Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 1 л."
1,242-12,Антигололед - 32 PROSEPTготовый состав / 12 кг
2,0024-06 с,"Герметик акриловый цвет сосна, ф/п 600мл"
3,305-2,"Кондиционер для белья ""Королевский Ирис"" Prosept Crystal Rinser, 2 л."
4,0024-7 б,NaN


Предобработка

In [8]:
data_products = data_products.dropna().reset_index()
data_dealers = data_dealers.dropna().reset_index()
data_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   index    485 non-null    int64 
 1   article  485 non-null    object
 2   name_1c  485 non-null    object
dtypes: int64(1), object(2)
memory usage: 11.5+ KB


In [9]:
import tfidf_matcher as tm

In [10]:
mama = tm.matcher(data_dealers['product_name'], data_products['name_1c'],5,3)

In [12]:
mama.head()

,Original Name,Lookup 1,Lookup 1 Confidence,Lookup 1 Index,Lookup 2,Lookup 2 Confidence,Lookup 2 Index,Lookup 3,Lookup 3 Confidence,Lookup 3 Index,Lookup 4,Lookup 4 Confidence,Lookup 4 Index,Lookup 5,Lookup 5 Confidence,Lookup 5 Index,all_mathing
0,"Средство универсальное Prosept Universal Spray, 500мл","Универсальное чистящее средство PROSEPT Universal Spray, 0.5 л.",0.69,477,"Профессиональное чистящее средство PROSEPT Universal Spray, 5 л.",0.59,332,Профессиональное универсальное моющее средство Universal Prof,0.50,435,"Профессиональное универсальное моющее средство Universal Prof, 1 л.",0.49,424,Универсальное моющее и чистящее средствоUniversal Spray+концентрат 1:20-1:100 / 5 л,0.43,167,"[Универсальное чистящее средство PROSEPT Universal Spray, 0.5 л., Профессиональное чистящее средство PROSEPT Universal Spray, 5 л., Профессиональное универсальное моющее средство Universal Prof, Профессиональное универсальное моющее средство Universal Prof, 1 л., Универсальное моющее и чистящее средствоUniversal Spray+концентрат 1:20-1:100 / 5 л]"
1,"Концентрат Prosept Multipower для мытья полов, цитрус 1л","Концентрат для мытья полов и стен Multipower ""Цитрус"" концентрат / 0,8 л",0.63,165,"Набор Концентрат для мытья полов и стен Multipower ""Цитрус"" 0,8 л 2 ШТ",0.57,83,"Средство для мытья полов PROSEPT Multipower FLOOR, 1л.",0.57,408,"Средство для мытья полов PROSEPT Multipower citrus, 1л.",0.56,410,"Средство для мытья полов PROSEPT Multipower ""Цитрус"", 2 штуки*0.8 л.",0.56,57,"[Концентрат для мытья полов и стен Multipower ""Цитрус"" концентрат / 0,8 л, Набор Концентрат для мытья полов и стен Multipower ""Цитрус"" 0,8 л 2 ШТ, Средство для мытья полов PROSEPT Multipower FLOOR, 1л., Средство для мытья полов PROSEPT Multipower citrus, 1л., Средство для мытья полов PROSEPT Multipower ""Цитрус"", 2 штуки*0.8 л.]"
2,"Средство для чистки люстр Prosept Universal Anti-dust, 500мл","Несмываемое средство для очистки люстр PROSEPT Universal Anti-dust, 0.5 л.",0.74,202,"Набор для люстр Anti-Dust спрей, 0.5 л, 2 шт",0.46,199,"Набор для люстр Anti-Dust спрей, 0.5 л, 3 шт",0.44,204,"Профессиональное чистящее средство PROSEPT Universal Spray, 5 л.",0.26,332,"Универсальное чистящее средство PROSEPT Universal Spray, 0.5 л.",0.26,477,"[Несмываемое средство для очистки люстр PROSEPT Universal Anti-dust, 0.5 л., Набор для люстр Anti-Dust спрей, 0.5 л, 2 шт, Набор для люстр Anti-Dust спрей, 0.5 л, 3 шт, Профессиональное чистящее средство PROSEPT Universal Spray, 5 л., Универсальное чистящее средство PROSEPT Universal Spray, 0.5 л.]"
3,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 023-05","Удалитель ржавчины PROSEPT RUST REMOVER, 0.5 л.",0.94,273,"Удалитель ржавчины PROSEPT RUST REMOVER, концентрат, 1 л.",0.86,455,"Удалитель ржавчины PROSEPT RUST REMOVER, концентрат, 5 л.",0.86,242,"Удалитель мха PROSEPT, 5 л.",0.23,184,"Удалитель мха PROSEPT, 1 л.",0.22,195,"[Удалитель ржавчины PROSEPT RUST REMOVER, 0.5 л., Удалитель ржавчины PROSEPT RUST REMOVER, концентрат, 1 л., Удалитель ржавчины PROSEPT RUST REMOVER, концентрат, 5 л., Удалитель мха PROSEPT, 5 л., Удалитель мха PROSEPT, 1 л.]"
4,Средство моющее для бани и сауны Prosept Multipower Wood 1л,"Моющее средство для бани и сауны PROSEPT Multipower Wood, 1 л.",0.91,239,"Набор для бани (Universal Wood, Multipower Wood)",0.49,189,"Набор Multipower Wood, 1 л, 2шт",0.46,234,"Антисептик для бани и сауны PROSEPT SAUNA, концентрат, 1 л.",0.39,49,"Антисептик для бани и сауны PROSEPT SAUNA, концентрат, 5 л.",0.39,376,"[Моющее средство для бани и сауны PROSEPT Multipower Wood, 1 л., Набор для бани (Universal Wood, Multipower Wood), Набор Multipower Wood, 1 л, 2шт, Антисептик для бани и сауны PROSEPT SAUNA, концентрат, 1 л., Антисептик для бани и сауны PROSEPT SAUNA, концентрат, 5 л.]"


In [19]:
mama['Original Name'][0]

'Средство универсальное Prosept Universal Spray, 500мл'

In [20]:
mama['all_mathing'][0]

['Универсальное чистящее средство PROSEPT Universal Spray, 0.5 л.',
 'Профессиональное чистящее средство PROSEPT Universal Spray, 5 л.',
 'Профессиональное универсальное моющее средство Universal Prof',
 'Профессиональное универсальное моющее средство Universal Prof, 1 л.',
 'Универсальное моющее и чистящее средствоUniversal Spray+концентрат 1:20-1:100 / 5 л']

In [21]:
def calculate_accuracy_a_5(y_true, y_pred_5_in_array):
    acc = 0
    for target, recommendations in zip(y_true.tolist(), y_pred_5_in_array.tolist()):
        acc += int(target in recommendations)
    return 100 * acc / len(y_pred_5_in_array)

In [22]:
calculate_accuracy_a_5(mama['Original Name'],mama['all_mathing'])

0.0

Не понял пока почему не считает метрику.